In [1]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [2]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

In [11]:
lp = StanProblem(bs_model);
D = 10;
initial_θ = rand(Uniform(-2, 2), D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.95, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

Sampling  19%|█████▊                         |  ETA: 0:00:01
  iterations:                                   1483
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.9106666264617739
  log_density:                                  -6.657185904792787
  hamiltonian_energy:                           12.01775908364423
  hamiltonian_energy_error:                     -0.15313000515186914
  max_hamiltonian_energy_error:                 0.37039605135905873
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.3654092796657946
  nom_step_size:                                0.3654092796657946
  is_adapt:                                     true



















Sampling  44%|█████████████▊                 |  ETA: 0:00:00
  iterations:                                   3557
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.7635501139470634
  log_density:                                  -5.813221905263495
  hamiltonian_energy:                           10.611802547968939
  hamiltonian_energy_error:                     0.14860118083220897
  max_hamiltonian_energy_error:                 0.4466739503531514
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.37954713739422064
  nom_step_size:                                0.37954713739422064
  is_adapt:                                     false



















Sampling  70%|█████████████████████▉         |  ETA: 0:00:00
  iterations:                                   5630
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.8887688560005993
  log_density:                                  -4.465182097858104
  hamiltonian_energy:                           9.000772413118657
  hamiltonian_energy_error:                     0.10627278100540138
  max_hamiltonian_energy_error:                 0.3359893575099875
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.37954713739422064
  nom_step_size:                                0.37954713739422064
  is_adapt:                                     false



















Sampling  91%|████████████████████████████▎  |  ETA: 0:00:00
  iterations:                                   7283
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.853552343901618
  log_density:                                  -6.186129378056484
  hamiltonian_energy:                           8.49810625829114
  hamiltonian_energy_error:                     0.18376668232929028
  max_hamiltonian_energy_error:                 0.19861723986777946
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.37954713739422064
  nom_step_size:                                0.37954713739422064
  is_adapt:                                     false



















Sampling 100%|███████████████████████████████| Time: 0:00:00
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.8306549728807495
  log_density:                                  -8.552853772561875
  hamiltonian_energy:                           12.967571734631452
  hamiltonian_energy_error:                     0.17636009485116233
  max_hamiltonian_energy_error:                 0.26574270445808246
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.37954713739422064
  nom_step_size:                                0.37954713739422064
  is_adapt:                                     false
  mass_matrix:                 

┌ Info: Finished 8000 sampling steps for 1 chains in 0.505972318 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([1.0139979627434632, 0.8752 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.38), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 0.9901998086984645
│   average_acceptance_rate = 0.9195218974616542
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [4]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
    param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
]...);

In [5]:
lol = hcat(samples...)'
conhmc = vcat([
    param_constrain(bs_model, collect(row))' for row in eachrow(lol)
]...);

In [6]:
mean(unc_sample_array, dims=1)

1×10 Matrix{Float64}:
 6.1505  4.93958  3.90591  4.79602  …  6.31717  4.884  4.41052  0.808081

In [7]:
mean(conhmc, dims=1)

1×10 Matrix{Float64}:
 0.323213  0.0841342  -0.0971426  0.0596639  …  0.0918739  4.4095  3.59251